The goal of this notebook is to create a student-teacher model where we first train a teacher on labeled data, and then use this teacher model to label more data, then we swap out the teacher with a student and train again over all the samples. 
- Try AutoAugment/RandAugment
- Add regularization
- Resampling  
  - Make resampling func for unlab_ds
- Create new unlab_ds without previous findings

# Loading data

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

import numpy as np
import datetime
import time
import os
import pathlib
import matplotlib.pyplot as plt
 
# Some stuff to make utils-function work
import sys
sys.path.append('../utils')
from pipeline import create_dataset, split_and_create_dataset, prepare_for_training
from utils import show_image, class_distribution, print_split_info, unpipe
%load_ext autoreload
%autoreload 2

# Jupyter-specific
%matplotlib inline

Some parameters

In [ ]:
project_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [ ]:
data_dir = pathlib.Path('/home/henriklg/master-thesis/data/hyper-kvasir/labeled_ttv/')
unlab_dir = pathlib.Path('/home/henriklg/master-thesis/data/hyper-kvasir/unlabeled_ttv/')
model_name = "teacher1"
log_dir = "./logs/{}/{}".format(project_time, model_name)

conf = {
    # Dataset
    "data_dir": data_dir,
    "unlab_dir": unlab_dir,
    "log_dir": log_dir,
    "cache_dir": "./cache",
    "ds_info": 'hypkva',
    "augment": ["crop","flip","brightness","saturation","contrast","rotate"],
    "aug_mult": 0.1,
    "resample": True,
    "class_weight": False,
    "shuffle_buffer_size": 2000,        # 0=no shuffling
    "seed": 2511,
    "neg_class": None,                 # select neg class for binary ds (normal class)
    "outcast": None,                   # list of folders to drop - currently only works for 1 item
    # Model
    "model_name": model_name,
    "model": 'EfficientNetB0',
    "dropout": 0.1,
    "num_epochs": 25,
    "batch_size": 128,
    "img_shape": (128, 128, 3),
    "learning_rate": 0.01,
    "optimizer": 'Adam',
    "final_activation": 'softmax',     # sigmoid for binary ds
    # Callbacks
    "learning_schedule": True,
    "decay_rate": 0.1,                 # higher number gives steeper lr dropoff
    "checkpoint": False,
    "early_stopp": True,
    "early_stopp_patience": 8,
    # Misc
    "verbosity": 1
    }

Create training, testing and validation dataset from utils/data_prep.py.  
Returns tf.dataset for shuffled, cached and batched data

In [ ]:
ds = create_dataset(conf)

class_names = conf["class_names"]

In [ ]:
from utils import checkout_dataset

# Show some images from training dataset - mainly to verify augmentation and distribution
# add params for title and log_dir for savefig
checkout_dataset(ds["train"], conf)

# Step 1: Train a teacher model on labeled images

In [ ]:
from tensorflow.python.keras.models import Model, load_model
from create_model import create_model

teacher_model = create_model(conf)

### Callbacks

In [ ]:
from create_model import create_callbacks

callbacks = create_callbacks(conf)

In [ ]:
from utils import write_to_file

write_to_file(conf, conf, "conf") #ignore stupid arguments

#### Class weight

In [ ]:
from utils import get_class_weights

class_weights = get_class_weights(ds["train"], conf)

### Train the teacher model

In [ ]:
start_time = time.time()

teacher_history = teacher_model.fit(
        ds["train"],
        steps_per_epoch = conf["steps"]["train"],
        epochs = conf["num_epochs"],
        validation_data = ds["test"],
        validation_steps = conf["steps"]["test"],
        validation_freq = 1,
        class_weight = class_weights,
        callbacks = callbacks,
        verbose = 1
)
print ("Time spent on training: {}".format(time.time() - start_time))

# Save the metrics from training
write_to_file(teacher_history.history, conf, "teacher_history")

### Save or restore a model

In [ ]:
teacher_model.save(conf["log_dir"]+'/model')

## Evaluate the model

In [ ]:
teacher_evaluate = teacher_model.evaluate(ds["val"], verbose=2, steps=conf["steps"]["val"])

write_to_file(teacher_evaluate, conf, "evaluate_val")

In [ ]:
from model_evaluation import plot_lr_and_accuracy

plot_lr_and_accuracy(teacher_history, conf)

In [ ]:
from model_evaluation import display_classification_report
from model_evaluation import get_metrics, get_confusion_matrix
from model_evaluation import show_dataset_predictions
from model_evaluation import plot_confusion_matrix

In [ ]:
eval_ds = unpipe(ds["val"], conf["ds_sizes"]["val"]).as_numpy_iterator()
eval_ds = np.array(list(eval_ds))
eval_images = np.stack(eval_ds[:,0], axis=0)

predictions = teacher_model.predict(eval_images, verbose=1)
pred_confidence = [np.max(pred) for pred in predictions]

true_labels = list(eval_ds[:,1])
pred_labels = [np.argmax(pred) for pred in predictions]

In [ ]:
get_metrics(true_labels, pred_labels)

In [ ]:
display_classification_report(
        true_labels, 
        pred_labels, 
        range(conf["num_classes"]), 
        target_names=conf["class_names"]
)

In [ ]:
cm = get_confusion_matrix(true_labels, pred_labels)

plot_confusion_matrix(cm, log_dir, conf["class_names"], figsize=(12,10))

Display grid of some random samples from validation data with the prediction confidence

In [ ]:
show_dataset_predictions(
        true_labels,
        pred_labels,
        pred_confidence,
        eval_images,
        conf,
)

# Step 2: use the teacher to generate pseudo labels on unlabeled images

### Read in the unlabeled dataset

In [ ]:
from pipeline import create_unlab_ds

unlab_ds, unlab_size = create_unlab_ds(conf)

## Run predictions on all unlabeled images
Using 'append to list and convert to tensor'-method

In [ ]:
from IPython.display import clear_output
from utils import print_bar_chart
from utils import get_tqdm

pred_confidence = 0.80
new_findings = 0
count = 0

pred_list = []
lab_list = []
name_list = []

In [ ]:
total_time = time.time()

tqdm_predicting, tqdm_findings = get_tqdm(unlab_size, count, new_findings)

print ("Press 'Interrupt Kernel' to save and exit.")
try:
    for count, (image,path) in enumerate(unlab_ds, start=count):
        img = np.expand_dims(image, 0)
        pred = teacher_model.predict(img)
        highest_pred = np.max(pred)
        if highest_pred > pred_confidence:
            pred_idx = np.argmax(pred).astype(np.uint8)

            lab_list.append(pred_idx)
            pred_list.append(highest_pred)
            name_list.append(path)
            
            # Clear old bar chart, generate new one and refresh the tqdm progress bars
            # NB, tqdm run-timer is also reset, unfortunately
            if not new_findings%500 and new_findings>100:
                clear_output(wait=True)
                tqdm_predicting, tqdm_findings = get_tqdm(unlab_size, count, new_findings)
                lab_array = np.asarray(lab_list, dtype=np.uint8)
                findings = np.bincount(lab_array, minlength=int(conf["num_classes"]))
                print_bar_chart([findings], conf)
                
            new_findings += 1
            tqdm_findings.update(1)
        tqdm_predicting.update(1)
except KeyboardInterrupt:
    print ("Exiting")

finally:
    print ("\nTotal run time: {:.3f} s".format( time.time() - total_time ))
    print ("Found {} new samples in unlabeled_ds after looking at {} images.".format(new_findings, count))

Plot bar chart and save it (optional)

In [ ]:
lab_array = np.asarray(lab_list, dtype=np.uint8)
findings = np.bincount(lab_array, minlength=int(conf["num_classes"]))

print_bar_chart(
    data=[findings],
    conf=conf,
    title=None,
    fname="bar_chart-findings_teacher",
    figsize=(16,7)
)
clear_output(wait=False)

Save the image and labels list as pickle dump (optional)

## Inspect the classified images

### Sort new samples after prediction confidence

In [ ]:
from utils import custom_sort

pred_list, lab_list, name_list = custom_sort(pred_list, lab_list, name_list)

### Display samples from all classes

In [ ]:
from utils import checkout_findings

unlabeled_findings = [pred_list, lab_list, name_list]
checkout_findings(unlabeled_findings, conf)

### Print grid of images from one of the classes

In [ ]:
from utils import checkout_class

checkout_class("pylorus", unlabeled_findings, conf)

### Resample the new findings from unlabeled dataset to "fit" original distribution of samples per class

In [ ]:
from utils import resample_unlab

new_findings, new_filepaths = resample_unlab(unlabeled_findings, ds["clean_train"], conf)

### Convert image and label lists to tensors and combine with training_ds to create a new dataset for training

In [ ]:
# create tf.tensor of the new findings
findings_tensor = tf.data.Dataset.from_tensor_slices(new_findings)

# combine with original training_ds (using clean_ds which is not augmented/repeated etc)
ds["psuedo_train"] = ds["clean_train"].concatenate(findings_tensor)

# count samples in the original and new/combined dataset
_, dist_teacher1 = class_distribution(ds["clean_train"], conf["num_classes"])
_, dist_student1 = class_distribution(ds["psuedo_train"], conf["num_classes"])

# History of class distribution
from utils import print_bar_chart
print_bar_chart(
    data=[dist_teacher1, dist_student1],
    conf=conf,
    title=None,
    fname="bar_chart-distribution2"
)

### 'Refresh' unlabeled dataset by extracting samples already used

In [ ]:
from utils import reduce_dataset

unlab_ds = reduce_dataset(unlab_ds, remove=new_filepaths)

unlab_size_teacher = unlab_size
unlab_size = unlab_size - len(new_filepaths)

### Sanity check

In [ ]:
print ("number of samples added to training data and removed from unlab_ds:", len(new_filepaths))

print ("\noriginal unlab_ds_size:", unlab_size_teacher)
print ("new unlab_ds_size:", unlab_size)

print ("\noriginal train_size:", int(np.sum(dist_teacher1)))
new_train_size = int(np.sum(dist_student1))
print ("new train dataset size:", new_train_size)

# Step 3: Train a student model on the combination of labeled images and pseudo labeled images

Now we have trained a teacher model, and used that model to predict on unlabeled dataset to create more samples with psudo-labels.  
It's time for swapping the teacher with the student!

In [ ]:
# save teacher conf
teacher_conf = conf

# Make changes
model_name = "student1"
log_dir = "./logs/{}/{}".format(project_time, model_name)

# Dataset
conf["log_dir"] = log_dir
conf["ds_sizes"]["train"] = new_train_size
conf["aug_mult"] = 0.5
# Model
conf["model_name"] = model_name
conf["model"] = 'EfficientNetB2'
conf["dropout"] = 0.3
conf["num_epochs"] = 25

In [ ]:
ds["train"] = prepare_for_training(
        ds=ds["psuedo_train"], 
        ds_name='train_stud1',
        num_classes=conf["num_classes"],
        conf=conf,
        cache=True
    )

In [ ]:
student_model = create_model(conf)

callbacks = create_callbacks(conf) 

class_weights = get_class_weights(ds["train"], conf)

write_to_file(conf, conf, "conf") 

In [ ]:
start_time = time.time()

stud_history = student_model.fit(
    ds["train"],
    steps_per_epoch = conf["steps"]["train"], 
    epochs = conf["num_epochs"],
    validation_data = ds["test"],
    validation_steps = conf["steps"]["test"],
    validation_freq = 1,
    callbacks = callbacks
)
print ("Time spent on training: {}".format(time.time() - start_time))

# Save the metrics from training
write_to_file(stud_history.history, conf, "student_history")

# Save the model
student_model.save(conf["log_dir"]+'/model')

## Evaluate the model

In [ ]:
eval_ds = unpipe(ds["val"], conf["ds_sizes"]["val"]).as_numpy_iterator()
eval_ds = np.array(list(eval_ds))
eval_images = np.stack(eval_ds[:,0], axis=0)

predictions = student_model.predict(eval_images, verbose=1)
pred_confidence = [np.max(pred) for pred in predictions]

true_labels = list(eval_ds[:,1])
pred_labels = [np.argmax(pred) for pred in predictions]

student_evaluate = student_model.evaluate(ds["val"], verbose=2, steps=conf["steps"]["val"])
write_to_file(student_evaluate, conf, "evaluate_val")

In [ ]:
plot_lr_and_accuracy(teacher_history, conf)
# get_metrics(true_labels, pred_labels)
display_classification_report(
        true_labels, 
        pred_labels, 
        range(conf["num_classes"]), 
        target_names=conf["class_names"]
)

cm = get_confusion_matrix(true_labels, pred_labels)
plot_confusion_matrix(cm, log_dir, conf["class_names"], figsize=(10,8))

show_dataset_predictions(
        true_labels,
        pred_labels,
        pred_confidence,
        eval_images,
        conf,
)

# Step 3.2: use the teacher to generate pseudo labels on unlabeled images

In [ ]:
pred_confidence = 0.80
new_findings = 0
count = 0

lab_list = []
pred_list = []
path_list = []

In [ ]:
total_time = time.time()

tqdm_predicting, tqdm_findings = get_tqdm(unlab_size, count, new_findings)

print ("Press 'Interrupt Kernel' to save and exit.")
try:
    for count, (image,path) in enumerate(unlab_ds, start=count):
        img = np.expand_dims(image, 0)
        pred = student_model.predict(img)
        highest_pred = np.max(pred)
        if highest_pred > pred_confidence:
            pred_idx = np.argmax(pred).astype(np.uint8)

            lab_list.append(pred_idx)
            pred_list.append(highest_pred)
            name_list.append(path)
            
            # Clear old bar chart, generate new one and refresh the tqdm progress bars
            # NB, tqdm run-timer is also reset, unfortunately
            if not new_findings%500 and new_findings>100:
                clear_output(wait=True)
                tqdm_predicting, tqdm_findings = get_tqdm(unlab_size, count, new_findings)
                lab_array = np.asarray(lab_list, dtype=np.uint8)
                findings = np.bincount(lab_array, minlength=int(conf["num_classes"]))
                print_bar_chart([findings], conf)
                
            new_findings += 1
            tqdm_findings.update(1)
        tqdm_predicting.update(1)
except KeyboardInterrupt:
    print ("Exiting")

finally:
    print ("\nTotal run time: {:.3f} s".format( time.time() - total_time ))
    print ("Found {} new samples in unlabeled_ds after looking at {} images.".format(new_findings, count))

In [ ]:
lab_array = np.asarray(lab_list, dtype=np.uint8)
findings = np.bincount(lab_array, minlength=int(conf["num_classes"]))

print_bar_chart(
    data=[findings],
    conf=conf,
    title=None,
    fname="bar_chart-findings_student",
    figsize=(16,7)
)
clear_output(wait=False)

### Inspect the classified images

In [ ]:
# Sort the new findings after confidence
pred_list, lab_list, name_list = custom_sort(pred_list, lab_list, name_list)

# Display samples for each class
unlab_findings = [pred_list, lab_list, name_list]
checkout_findings(unlab_findings, conf)

In [ ]:
# Print grid of images from one of the classes
checkout_class("pylorus", unlab_findings, conf)

### Prepare new training and unlabeled datasets

In [ ]:
# Resample the new findings to original distribution
# Create tf.tensors of new findings
if conf["resample"]:
    new_findings, added_samples = resample_unlab(unlab_findings, ds["psuedo_train"], conf)
    findings_tensor = tf.data.Dataset.from_tensor_slices(new_findings)
else:
    added_samples = len(name_list)
    img_list = [fn2img(name, conf["unlab_ds"], conf["img_shape"][0]) for name in name_list]
    findings_tensor = tf.data.Dataset.from_tensor_slices([img_list, lab_list])

# combine with previous training dataset
ds["psuedo_train"] = ds["psuedo_train"].concatenate(findings_tensor)

# count samples in the original and new/combined dataset
_, dist_teacher2 = class_distribution(ds["psuedo_train"], conf["num_classes"])

# Display history of class distribution
print_bar_chart(
    data=[dist_teacher1, dist_student1, dist_teacher2],
    conf=conf,
    title=None,
    fname="bar_chart-distribution3"
)

# Refresh the unlabeled dataset
unlab_ds = reduce_dataset(unlab_ds, remove=new_filepaths)

unlab_size_student = unlab_size
unlab_size = unlab_size - len(added_samples)

### Sanity check

In [ ]:
print ("number of samples added to training data and removed from unlab_ds:", len(new_filepaths))

print ("\noriginal unlab_ds_size:", unlab_size_teacher)
print ("new unlab_ds_size:", unlab_size)

print ("\noriginal train_size:", int(np.sum(dist_teacher1)))
print ("new train dataset size:", int(np.sum(dist_student1)))

# Step 4: Iterate this algorithm a few times by treating the student as a teacher to relabel the unlabeled data and training a new student